In [0]:
#!import "../Utils/Queries"
#!import "../Report/ReportConfigurationAndUtils"
#!import "TestData"

# Initialization

In [0]:
await DataSource.UpdateAsync( new[] { partitionReportingNode } );
await DataSource.UpdateAsync( new[] { partition, previousPeriodPartition, partitionScenarioMTUP } );
await DataSource.UpdateAsync( new[] { dt11 } );

# Import

## Data

In [0]:
var bestEstimateRawVars = new[] { new RawVariable { AmountType = "PR", AocType = "CL", Novelty = "I", Partition = partition.Id, Values = new[] {1.0} }, 
                                  new RawVariable { AmountType = "PR", AocType = "AU", Novelty = "I", Partition = partition.Id, Values = new[] {2.0} },
                                  new RawVariable { AmountType = "PR", AocType = "EV", Novelty = "I", Partition = partition.Id, Values = new[] {3.0} },
                                  new RawVariable { AmountType = "CL", AocType = "CL", Novelty = "I", Partition = partition.Id, Values = new[] {4.0} },
                                  new RawVariable { AmountType = "CL", AocType = "AU", Novelty = "I", Partition = partition.Id, Values = new[] {5.0} },
                                  new RawVariable { AmountType = "CL", AocType = "EV", Novelty = "I", Partition = partition.Id, Values = new[] {6.0} } };

In [0]:
var scenarioRawVars = new[] { new RawVariable { AmountType = "PR", AocType = "CL", Novelty = "I", Partition = partitionScenarioMTUP.Id, Values = new[] {1.1} }, 
                              new RawVariable { AmountType = "PR", AocType = "AU", Novelty = "I", Partition = partitionScenarioMTUP.Id, Values = new[] {2.1} } };

In [0]:
var newScenarioRawVars = new[] { new RawVariable { AmountType = "PR", AocType = "CL", Novelty = "I", Partition = partitionScenarioMTUP.Id, Values = new[] {11.0} }, 
                                 new RawVariable { AmountType = "CL", AocType = "CL", Novelty = "I", Partition = partitionScenarioMTUP.Id, Values = new[] {41.0} } };

## Scenario Query and Workspace with scenario data

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<RawVariable>().ToArrayAsync() );
await DataSource.UpdateAsync( bestEstimateRawVars.Concat(scenarioRawVars).ToArray() );
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
await ws.UpdateAsync(newScenarioRawVars);
var queriedRawVars = await ws.QueryPartitionedDataAsync<RawVariable,PartitionByReportingNodeAndPeriod>(DataSource, partitionScenarioMTUP.Id, partition.Id, ImportFormats.Cashflow);

In [0]:
queriedRawVars.SelectMany(x => x.Values).Sum().Should().Be(52.0);

## Scenario query, Workspace with no new scenario data, mismatching import format

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<RawVariable>().ToArrayAsync() );
await DataSource.UpdateAsync( bestEstimateRawVars.Concat(scenarioRawVars).ToArray() );
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
var queriedRawVars = await ws.QueryPartitionedDataAsync<RawVariable,PartitionByReportingNodeAndPeriod>(DataSource, partitionScenarioMTUP.Id, partition.Id, ImportFormats.Actual);

In [0]:
queriedRawVars.SelectMany(x => x.Values).Sum().Should().Be(0);

## Scenario query, Workspace with no new scenario data and DataSource with scenario data

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<RawVariable>().ToArrayAsync() );
await DataSource.UpdateAsync( bestEstimateRawVars.Concat(scenarioRawVars).ToArray() );
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
var queriedRawVars = await ws.QueryPartitionedDataAsync<RawVariable,PartitionByReportingNodeAndPeriod>(DataSource, partitionScenarioMTUP.Id, partition.Id, ImportFormats.Cashflow);

In [0]:
queriedRawVars.SelectMany(x => x.Values).Sum().Should().Be(3.2);

## Scenario query, Workspace with no new scenario data and DataSource with only Best Estimate data

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<RawVariable>().ToArrayAsync() );
await DataSource.UpdateAsync( bestEstimateRawVars.ToArray() );
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
var queriedRawVars = await ws.QueryPartitionedDataAsync<RawVariable,PartitionByReportingNodeAndPeriod>(DataSource, partitionScenarioMTUP.Id, partition.Id, ImportFormats.Cashflow);

In [0]:
queriedRawVars.SelectMany(x => x.Values).Sum().Should().Be(21);

# Report

## Data

In [0]:
var bestEstimateIfrsVars = new[] { new IfrsVariable { DataNode = "DT1.1", AmountType = "PR", AocType = "CL", Novelty = "I", Partition = partition.Id, Value = 1.0 }, 
                                   new IfrsVariable { DataNode = "DT1.1", AmountType = "PR", AocType = "AU", Novelty = "I", Partition = partition.Id, Value = 2.0 },
                                   new IfrsVariable { DataNode = "DT1.1", AmountType = "PR", AocType = "EV", Novelty = "I", Partition = partition.Id, Value = 3.0 },
                                   new IfrsVariable { DataNode = "DT1.1", AmountType = "CL", AocType = "CL", Novelty = "I", Partition = partition.Id, Value = 4.0 },
                                   new IfrsVariable { DataNode = "DT1.1", AmountType = "CL", AocType = "AU", Novelty = "I", Partition = partition.Id, Value = 5.0 },
                                   new IfrsVariable { DataNode = "DT1.1", AmountType = "CL", AocType = "EV", Novelty = "I", Partition = partition.Id, Value = 6.0 } };

In [0]:
var scenarioIfrsVars = new[] { new IfrsVariable { DataNode = "DT1.1", AmountType = "PR", AocType = "CL", Novelty = "I", Partition = partitionScenarioMTUP.Id, Value = 1.1 }, 
                               new IfrsVariable { DataNode = "DT1.1", AmountType = "PR", AocType = "AU", Novelty = "I", Partition = partitionScenarioMTUP.Id, Value = 2.1 } };

## Best estimate query

In [0]:
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
await ws.UpdateAsync(bestEstimateIfrsVars);
await ws.UpdateAsync(scenarioIfrsVars);

In [0]:
(await ws.QueryReportVariablesAsync((args.Year, args.Month, args.ReportingNode, args.Scenario))).Select(x => x.Value).Sum().Should().Be(21);

## Scenario query

In [0]:
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
await ws.UpdateAsync(bestEstimateIfrsVars);
await ws.UpdateAsync(scenarioIfrsVars);

In [0]:
(await ws.QueryReportVariablesAsync((args.Year, args.Month, args.ReportingNode, argsScenarioMTUP.Scenario))).Select(x => x.Value).Sum().Should().Be(21.2);